In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import re
import io
import os
import requests
import unicodedata
from torch.utils.data import Dataset, DataLoader

# Check for a GPU and use it if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the number of examples to use from the dataset.
num_examples = 10000
embedding_dim = 256
hidden_size = 1024

def download_data():
    """
    Downloads a small English-Spanish parallel corpus from a reliable source.

    Returns:
        str: The file path to the extracted text file.
    """
    print("Downloading dataset...")
    # Using a reliable raw text file from the Tatoeba project via a stable GitHub source
    url = "/content/deu_update.txt"
    try:
        #response = requests.get(url)
        #response.raise_for_status()  # Raise an exception for bad status codes

        file_path = 'eng-spa.txt'
        with open(url, 'wb') as f:
            f.write(response.content)

        print(f"Dataset downloaded to {file_path}")
        return file_path
    except requests.exceptions.RequestException as e:
        print(f"Error downloading the file: {e}")
        return None

def preprocess_sentence(sentence):
    """
    Preprocesses a sentence by cleaning the text and adding start/end tokens.

    Args:
        sentence (str): The raw sentence string.

    Returns:
        str: The preprocessed sentence.
    """
    # Normalize unicode characters to simplify the text
    sentence = ''.join(c for c in unicodedata.normalize('NFD', sentence) if unicodedata.category(c) != 'Mn')

    sentence = sentence.lower().strip()
    # Create a space between a word and the punctuation following it
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # Replace all characters except letters, punctuation, and spaces
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)

    sentence = sentence.strip()

    # Add start and end tokens
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

def create_dataset(path, num_examples):
    """
    Creates a dataset from a text file, limiting it to a specific number of examples.

    Args:
        path (str): The file path to the parallel corpus.
        num_examples (int): The number of sentence pairs to load.

    Returns:
        tuple: A tuple containing lists of preprocessed input and target sentences.
    """
    print("Creating dataset from file...")
    with io.open(path, encoding='UTF-8') as f:
        lines = f.read().strip().split('\n')

    # The dataset format is "English_sentence\tSpanish_sentence"
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')] for l in lines[:num_examples]]

    # We'll use Spanish as the input and English as the target.
    # The pairs are stored as (English, Spanish) so we swap them
    inp_lang = [pair[1] for pair in word_pairs]
    targ_lang = [pair[0] for pair in word_pairs]

    return inp_lang, targ_lang

# --- Model Components ---

class Encoder(nn.Module):
    def __init__(self, input_vocab_size, embedding_dim, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, self.hidden_size, batch_first=True)

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.W1 = nn.Linear(hidden_size, hidden_size)
        self.W2 = nn.Linear(hidden_size, hidden_size)
        self.V = nn.Linear(hidden_size, 1)

    def forward(self, query, values):
        # query_with_time_axis shape: (batch_size, 1, hidden_size)
        query_with_time_axis = query.unsqueeze(1)

        # score shape: (batch_size, max_length, 1)
        score = self.V(torch.tanh(self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape: (batch_size, max_length, 1)
        attention_weights = torch.softmax(score, dim=1)

        # context_vector shape: (batch_size, hidden_size)
        context_vector = torch.sum(attention_weights * values, dim=1)

        return context_vector, attention_weights

class Decoder(nn.Module):
    def __init__(self, output_vocab_size, embedding_dim, hidden_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim + hidden_size, self.hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_vocab_size)
        self.attention = BahdanauAttention(self.hidden_size)

    def forward(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after embedding: (batch_size, 1, embedding_dim)
        embedded = self.embedding(x)

        # concatenated_input shape: (batch_size, 1, embedding_dim + hidden_size)
        concatenated_input = torch.cat((context_vector.unsqueeze(1), embedded), dim=-1)

        output, hidden = self.gru(concatenated_input, hidden.unsqueeze(0))

        # output shape: (batch_size * 1, hidden_size)
        output = output.squeeze(1)

        # predictions shape: (batch_size, output_vocab_size)
        predictions = self.fc(output)

        return predictions, hidden.squeeze(0), attention_weights

# --- Training and Translation Functions ---

def train_step(encoder, decoder, optimizer, criterion, input_tensor, target_tensor, max_length_target, targ_lang_tokenizer):
    encoder_hidden = encoder.init_hidden(input_tensor.size(0))
    encoder_output, encoder_hidden = encoder(input_tensor, encoder_hidden)

    decoder_input = torch.tensor([[targ_lang_tokenizer['word_to_idx']['<start>']]] * input_tensor.size(0), device=device)
    decoder_hidden = encoder_hidden.squeeze(0)

    loss = 0
    for t in range(target_tensor.size(1)):
        predictions, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_output)

        # Mask out padding tokens from loss calculation
        mask = target_tensor[:, t].ne(0).float()
        loss += criterion(predictions, target_tensor[:, t].to(device)) * mask

        decoder_input = target_tensor[:, t].unsqueeze(1)

    # Calculate average loss per sequence
    loss /= target_tensor.size(1)

    optimizer.zero_grad()
    loss.sum().backward()
    optimizer.step()

    return loss.mean().item()

def translate(sentence, encoder, decoder, inp_lang_tokenizer, targ_lang_tokenizer, max_length_input, max_length_target):
    encoder.eval()
    decoder.eval()

    sentence = preprocess_sentence(sentence)

    try:
        inputs = [inp_lang_tokenizer['word_to_idx'][i] for i in sentence.split(' ')]
    except KeyError as e:
        return f"Error: Word '{e}' not found in Spanish vocabulary. Please try a different sentence."

    inputs = torch.tensor(inputs, device=device).unsqueeze(0)

    encoder_hidden = encoder.init_hidden(1)
    encoder_output, encoder_hidden = encoder(inputs, encoder_hidden)

    decoder_hidden = encoder_hidden.squeeze(0)
    decoder_input = torch.tensor([[targ_lang_tokenizer['word_to_idx']['<start>']]], device=device)

    translated_words = []

    for _ in range(max_length_target):
        predictions, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_output)

        predicted_id = predictions.argmax(1).item()
        translated_word = targ_lang_tokenizer['idx_to_word'][predicted_id]

        if translated_word == '<end>':
            break

        translated_words.append(translated_word)
        decoder_input = torch.tensor([[predicted_id]], device=device)

    encoder.train()
    decoder.train()

    return ' '.join(translated_words)

# --- Custom Dataset and Vocab for PyTorch ---

class TranslationDataset(Dataset):
    def __init__(self, inp_lang, targ_lang):
        self.inp_lang = inp_lang
        self.targ_lang = targ_lang

        # Manually build the vocabularies to avoid torchtext issues
        self.inp_vocab = self.build_vocabulary(self.inp_lang)
        self.targ_vocab = self.build_vocabulary(self.targ_lang)

    def build_vocabulary(self, sentences):
        """
        Builds a word-to-index and index-to-word mapping from a list of sentences.
        """
        word_to_idx = {'<pad>': 0, '<unk>': 1, '<start>': 2, '<end>': 3}
        idx_to_word = {0: '<pad>', 1: '<unk>', 2: '<start>', 3: '<end>'}
        idx = 4

        for sentence in sentences:
            for word in sentence.split():
                if word not in word_to_idx:
                    word_to_idx[word] = idx
                    idx_to_word[idx] = word
                    idx += 1

        return {'word_to_idx': word_to_idx, 'idx_to_word': idx_to_word, 'size': len(word_to_idx)}

    def __len__(self):
        return len(self.inp_lang)

    def __getitem__(self, idx):
        return self.inp_lang[idx], self.targ_lang[idx]

def pad_collate_fn(batch, inp_vocab, targ_vocab):
    inp_list = []
    targ_list = []

    for inp, targ in batch:
        inp_list.append([inp_vocab['word_to_idx'].get(word, inp_vocab['word_to_idx']['<unk>']) for word in inp.split()])
        targ_list.append([targ_vocab['word_to_idx'].get(word, targ_vocab['word_to_idx']['<unk>']) for word in targ.split()])

    # PyTorch's pad_sequence pads to the longest sequence in the batch
    padded_inp = nn.utils.rnn.pad_sequence([torch.tensor(x, dtype=torch.long) for x in inp_list], batch_first=True, padding_value=inp_vocab['word_to_idx']['<pad>'])
    padded_targ = nn.utils.rnn.pad_sequence([torch.tensor(x, dtype=torch.long) for x in targ_list], batch_first=True, padding_value=targ_vocab['word_to_idx']['<pad>'])

    return padded_inp, padded_targ

if __name__ == '__main__':
    # 1. Prepare the dataset
    #file_path = download_data()
    #if file_path is None:
    #    print("Could not download the dataset. Exiting.")
    #else:
    if 1:
        inp_lang, targ_lang = create_dataset('/content/deu_update.txt', num_examples)

        # Create a PyTorch Dataset and DataLoader
        dataset = TranslationDataset(inp_lang, targ_lang)
        inp_vocab = dataset.inp_vocab
        targ_vocab = dataset.targ_vocab
        inp_vocab_size = inp_vocab['size']
        targ_vocab_size = targ_vocab['size']

        BATCH_SIZE = 64
        data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True,
                                 collate_fn=lambda x: pad_collate_fn(x, inp_vocab, targ_vocab))

        max_length_targ = max(len(s.split()) for s in targ_lang)
        max_length_inp = max(len(s.split()) for s in inp_lang)

        print("\nData preparation complete.")
        print("Max length of input sentences:", max_length_inp)
        print("Max length of target sentences:", max_length_targ)
        print("Size of input vocabulary:", inp_vocab_size)
        print("Size of target vocabulary:", targ_vocab_size)

        # 2. Build the model
        print("\nBuilding the model...")
        encoder = Encoder(inp_vocab_size, embedding_dim, hidden_size).to(device)
        decoder = Decoder(targ_vocab_size, embedding_dim, hidden_size).to(device)

        optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()))
        criterion = nn.CrossEntropyLoss(ignore_index=targ_vocab['word_to_idx']['<pad>'], reduction='none')

        # 3. Train the model
        print("Training the model...")
        EPOCHS = 5
        for epoch in range(EPOCHS):
            total_loss = 0
            for batch_idx, (input_tensor, target_tensor) in enumerate(data_loader):
                input_tensor = input_tensor.to(device)
                target_tensor = target_tensor.to(device)

                loss = train_step(encoder, decoder, optimizer, criterion, input_tensor, target_tensor, max_length_targ, targ_vocab)
                total_loss += loss

                if batch_idx % 10 == 0:
                    print(f"Epoch {epoch + 1}, Batch {batch_idx}, Loss: {loss:.4f}")

            print(f"Epoch {epoch + 1} finished. Avg Loss: {total_loss / len(data_loader):.4f}")

        print("\nTraining complete.")

        # 4. Save the model
        # Create a folder to save the models if it doesn't exist
        os.makedirs('models', exist_ok=True)

        torch.save(encoder.state_dict(), 'models/encoder.pth')
        torch.save(decoder.state_dict(), 'models/decoder.pth')

        print("Model saved successfully as 'models/encoder.pth' and 'models/decoder.pth'")

        # 5. Demonstrate translation
        print("\nDemonstrating translation of example sentences:")
        example_sentences = [
            'Kennen Sie sie',
            'Sehen Sie das',
            'Zählt das'
        ]

        for sentence in example_sentences:
            translated_sentence = translate(sentence, encoder, decoder, inp_vocab, targ_vocab, max_length_inp, max_length_targ)
            print(f"\nInput: {sentence}")
            print(f"Translated: {translated_sentence}")

Creating dataset from file...

Data preparation complete.
Max length of input sentences: 13
Max length of target sentences: 9
Size of input vocabulary: 3531
Size of target vocabulary: 2209

Building the model...
Training the model...
Epoch 1, Batch 0, Loss: 5.8860
Epoch 1, Batch 10, Loss: 2.4905
Epoch 1, Batch 20, Loss: 2.5202
Epoch 1, Batch 30, Loss: 2.2795
Epoch 1, Batch 40, Loss: 2.0400
Epoch 1, Batch 50, Loss: 1.5770
Epoch 1, Batch 60, Loss: 1.7602
Epoch 1, Batch 70, Loss: 1.5914
Epoch 1, Batch 80, Loss: 1.7128
Epoch 1, Batch 90, Loss: 1.4655
Epoch 1, Batch 100, Loss: 1.5485
Epoch 1, Batch 110, Loss: 1.4124
Epoch 1, Batch 120, Loss: 1.3748
Epoch 1, Batch 130, Loss: 1.2541
Epoch 1, Batch 140, Loss: 1.4025
Epoch 1, Batch 150, Loss: 1.4120
Epoch 1 finished. Avg Loss: 1.8165
Epoch 2, Batch 0, Loss: 1.0287
Epoch 2, Batch 10, Loss: 0.9546
Epoch 2, Batch 20, Loss: 0.9630
Epoch 2, Batch 30, Loss: 1.0273
Epoch 2, Batch 40, Loss: 0.9417
Epoch 2, Batch 50, Loss: 0.9862
Epoch 2, Batch 60, Loss

In [3]:
# 5. Demonstrate translation
print("\nDemonstrating translation of example sentences:")
example_sentences = [
    'Kennen Sie sie',
    'Sehen Sie das',
    'Zählt das'
]

for sentence in example_sentences:
    translated_sentence = translate(sentence, encoder, decoder, inp_vocab, targ_vocab, max_length_inp, max_length_targ)
    print(f"\nInput: {sentence}")
    print(f"Translated: {translated_sentence}")


Demonstrating translation of example sentences:

Input: Kennen Sie sie
Translated: <start> do you know .

Input: Sehen Sie das
Translated: <start> take that .

Input: Zählt das
Translated: <start> take this .
